In [1]:
magma_dir = '/home/ubuntu/magma/'
bucket_dir = '/home/ubuntu/s3/'
transformers_dir = '/home/ubuntu/transformers/'
cache_dir = bucket_dir+'.cache/'

## **Fine-tuning**

In [2]:
finetune_script = '"'+transformers_dir+'examples/seq2seq/finetune_trainer.py"'
eval_script = '"'+transformers_dir+'examples/seq2seq/run_eval.py"'

### **Config**

In [3]:
import sys
sys.path.insert(0, magma_dir)
import config

import torch
torch.manual_seed = config.SEED

import wandb
wandb.login()

project_name = 'hps_bart_para_wordembed'
%env WANDB_PROJECT=$project_name

Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable
wandb: Currently logged in as: marcoabrate (use `wandb login --relogin` to force relogin)


env: WANDB_PROJECT=hps_bart_para_wordembed


### HP search

In [4]:
model_name_or_path = 'sshleifer/distilbart-cnn-12-6'

In [5]:
data_dir = '"'+bucket_dir+'datasets/karger_books_para_wordembed/bart/st/"'

output_dir = '"'+bucket_dir+'fine-tuning/hps_bart_para_wordembed"'

log_dir = output_dir + '/logs'

In [6]:
hp_search_checkpoint_dir = output_dir + 'ray_checkpoints'

hp_search_log_dir = output_dir + 'ray_logs'

In [7]:
from transformers import AutoConfig
model_config = AutoConfig.from_pretrained(model_name_or_path, use_cache=False)
model_config.min_length = config.ONE_BULLET_MIN_LEN
model_config.max_length = config.ONE_BULLET_MAX_LEN

model_config.task_specific_params['summarization']['min_length'] = config.ONE_BULLET_MIN_LEN
model_config.task_specific_params['summarization']['max_length'] = config.ONE_BULLET_MAX_LEN
model_config_dir = '"'+bucket_dir+'fine-tuning/'+\
    model_name_or_path.replace('/', '?')+'_config"'
model_config.save_pretrained(model_config_dir[1:-1])

In [8]:
'''
--logging_steps 20 --logging_first_step \
--evaluation_strategy steps --eval_steps 50 --eval_beams 3 \
'''

'\n--logging_steps 20 --logging_first_step --evaluation_strategy steps --eval_steps 50 --eval_beams 3 '

In [10]:
!python3 $finetune_script \
--model_name_or_path $model_name_or_path \
--config_name $model_config_dir \
--tokenizer_name $model_name_or_path \
--cache_dir $cache_dir \
--data_dir $data_dir \
--freeze_embeds \
--hp_search \
--hp_search_checkpoint_dir $hp_search_checkpoint_dir --hp_search_local_mode --hp_search_db_host '0.0.0.0' \
--hp_search_name $project_name --hp_search_log_dir $hp_search_log_dir \
--lr_scheduler constant \
--warmup_steps 0 \
--fp16 \
--label_smoothing_factor 0.1 \
--sortish_sampler \
--task summarization \
--max_target_length $config.ONE_BULLET_MAX_LEN \
--val_max_target_length $config.ONE_BULLET_MAX_LEN \
--n_train 50 --n_val 10 \
--max_steps 10 \
--logging_steps 1 --logging_first_step \
--per_device_train_batch_size 1 --per_device_eval_batch_size 1 \
--evaluation_strategy steps --eval_steps 3 --eval_beams 3 \
--predict_with_generate \
--save_steps 50 \
--output_dir $output_dir \
--overwrite_output_dir \
--seed $config.SEED \
--run_name $output_dir

02/10/2021 14:20:12 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: True
02/10/2021 14:20:12 - INFO - __main__ -   Training/evaluation parameters Seq2SeqTrainingArguments(output_dir='/home/ubuntu/s3/fine-tuning/hps_bart_para_wordembed', overwrite_output_dir=True, do_train=False, do_eval=None, do_predict=False, evaluation_strategy=<EvaluationStrategy.STEPS: 'steps'>, prediction_loss_only=False, per_device_train_batch_size=1, per_device_eval_batch_size=1, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=3.0, max_steps=10, lr_scheduler_type=<SchedulerType.LINEAR: 'linear'>, warmup_steps=0, logging_dir='runs/Feb10_14-20-12_ip-172-31-39-35', logging_first_step=True, logging_steps=1, save_steps=50, save_total_limit=None, no_cu

[INFO|modeling_utils.py:1143] 2021-02-10 14:20:44,153 >> All model checkpoint weights were used when initializing BartForConditionalGeneration.

[INFO|modeling_utils.py:1152] 2021-02-10 14:20:44,153 >> All the weights of BartForConditionalGeneration were initialized from the model checkpoint at sshleifer/distilbart-cnn-12-6.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BartForConditionalGeneration for predictions without further training.
02/10/2021 14:20:44 - INFO - utils -   setting model.config to task specific params for summarization:
 {'early_stopping': True, 'length_penalty': 2.0, 'max_length': 142, 'min_length': 56, 'no_repeat_ngram_size': 3, 'num_beams': 4}
02/10/2021 14:20:44 - INFO - utils -   note: command line args may override some of these
[INFO|modeling_utils.py:1027] 2021-02-10 14:20:45,299 >> loading weights file https://huggingface.co/sshleifer/distilbart-cnn-12-6/resolve/main/pytorch_model.bin from cache at /hom

02/10/2021 14:22:32 - INFO - gensim.models.base_any2vec -   training model with 3 workers on 58 vocabulary and 100 features, using sg=1 hs=0 sample=0.001 negative=5 window=5
02/10/2021 14:22:32 - INFO - gensim.models.base_any2vec -   worker thread finished; awaiting finish of 2 more threads
02/10/2021 14:22:32 - INFO - gensim.models.base_any2vec -   worker thread finished; awaiting finish of 1 more threads
02/10/2021 14:22:32 - INFO - gensim.models.base_any2vec -   worker thread finished; awaiting finish of 0 more threads
02/10/2021 14:22:32 - INFO - gensim.models.base_any2vec -   EPOCH - 1 : training on 76 raw words (24 effective words) took 0.0s, 15787 effective words/s
02/10/2021 14:22:32 - INFO - gensim.models.base_any2vec -   worker thread finished; awaiting finish of 2 more threads
02/10/2021 14:22:32 - INFO - gensim.models.base_any2vec -   worker thread finished; awaiting finish of 1 more threads
02/10/2021 14:22:32 - INFO - gensim.models.base_any2vec -   worker thread finished;

02/10/2021 14:22:32 - INFO - gensim.models.base_any2vec -   worker thread finished; awaiting finish of 2 more threads
02/10/2021 14:22:32 - INFO - gensim.models.base_any2vec -   worker thread finished; awaiting finish of 1 more threads
02/10/2021 14:22:32 - INFO - gensim.models.base_any2vec -   worker thread finished; awaiting finish of 0 more threads
02/10/2021 14:22:32 - INFO - gensim.models.base_any2vec -   EPOCH - 5 : training on 69 raw words (16 effective words) took 0.0s, 51118 effective words/s
02/10/2021 14:22:32 - INFO - gensim.models.base_any2vec -   training on a 345 raw words (78 effective words) took 0.0s, 10058 effective words/s
02/10/2021 14:22:32 - WARNING - gensim.models.base_any2vec -   under 10 jobs per worker: consider setting a smaller `batch_words' for smoother alpha decay
02/10/2021 14:22:32 - INFO - gensim.models.word2vec -   collecting all words and their counts
02/10/2021 14:22:32 - INFO - gensim.models.word2vec -   PROGRESS: at sentence #0, processed 0 words,

02/10/2021 14:22:32 - INFO - gensim.models.base_any2vec -   training model with 3 workers on 60 vocabulary and 100 features, using sg=1 hs=0 sample=0.001 negative=5 window=5
02/10/2021 14:22:32 - INFO - gensim.models.base_any2vec -   worker thread finished; awaiting finish of 2 more threads
02/10/2021 14:22:32 - INFO - gensim.models.base_any2vec -   worker thread finished; awaiting finish of 1 more threads
02/10/2021 14:22:32 - INFO - gensim.models.base_any2vec -   worker thread finished; awaiting finish of 0 more threads
02/10/2021 14:22:32 - INFO - gensim.models.base_any2vec -   EPOCH - 1 : training on 97 raw words (30 effective words) took 0.0s, 65642 effective words/s
02/10/2021 14:22:32 - INFO - gensim.models.base_any2vec -   worker thread finished; awaiting finish of 2 more threads
02/10/2021 14:22:32 - INFO - gensim.models.base_any2vec -   worker thread finished; awaiting finish of 1 more threads
02/10/2021 14:22:32 - INFO - gensim.models.base_any2vec -   worker thread finished;

...

02/10/2021 14:25:44 - INFO - gensim.models.base_any2vec -   training model with 3 workers on 47 vocabulary and 100 features, using sg=1 hs=0 sample=0.001 negative=5 window=5
02/10/2021 14:25:44 - INFO - gensim.models.base_any2vec -   worker thread finished; awaiting finish of 2 more threads
02/10/2021 14:25:44 - INFO - gensim.models.base_any2vec -   worker thread finished; awaiting finish of 1 more threads
02/10/2021 14:25:44 - INFO - gensim.models.base_any2vec -   worker thread finished; awaiting finish of 0 more threads
02/10/2021 14:25:44 - INFO - gensim.models.base_any2vec -   EPOCH - 1 : training on 63 raw words (23 effective words) took 0.0s, 46555 effective words/s
02/10/2021 14:25:44 - INFO - gensim.models.base_any2vec -   worker thread finished; awaiting finish of 2 more threads
02/10/2021 14:25:44 - INFO - gensim.models.base_any2vec -   worker thread finished; awaiting finish of 1 more threads
02/10/2021 14:25:44 - INFO - gensim.models.base_any2vec -   worker thread finished;

02/10/2021 14:25:44 - INFO - gensim.models.base_any2vec -   worker thread finished; awaiting finish of 1 more threads
02/10/2021 14:25:44 - INFO - gensim.models.base_any2vec -   worker thread finished; awaiting finish of 0 more threads
02/10/2021 14:25:44 - INFO - gensim.models.base_any2vec -   EPOCH - 3 : training on 64 raw words (15 effective words) took 0.0s, 30025 effective words/s
02/10/2021 14:25:44 - INFO - gensim.models.base_any2vec -   worker thread finished; awaiting finish of 2 more threads
02/10/2021 14:25:44 - INFO - gensim.models.base_any2vec -   worker thread finished; awaiting finish of 1 more threads
02/10/2021 14:25:44 - INFO - gensim.models.base_any2vec -   worker thread finished; awaiting finish of 0 more threads
02/10/2021 14:25:44 - INFO - gensim.models.base_any2vec -   EPOCH - 4 : training on 64 raw words (13 effective words) took 0.0s, 31503 effective words/s
02/10/2021 14:25:44 - INFO - gensim.models.base_any2vec -   worker thread finished; awaiting finish of 2


wandb: Waiting for W&B process to finish, PID 6148
wandb: Program ended successfully.
wandb:                                                                                
wandb: Find user logs for this run at: /home/ubuntu/s3/fine-tuning/hps_bart_para_wordembedray_logs/hps_bart_para_wordembed/_inner_40086_00000_0_gradient_accumulation_steps=2,learning_rate=1e-05_2021-02-10_14-21-22/wandb/run-20210210_142445-3b3ncyu5/logs/debug.log
wandb: Find internal logs for this run at: /home/ubuntu/s3/fine-tuning/hps_bart_para_wordembedray_logs/hps_bart_para_wordembed/_inner_40086_00000_0_gradient_accumulation_steps=2,learning_rate=1e-05_2021-02-10_14-21-22/wandb/run-20210210_142445-3b3ncyu5/logs/debug-internal.log
wandb: Run summary:
wandb:                           train/loss 2.9814
wandb:                  train/learning_rate 5e-05
wandb:                          train/epoch 0.4
wandb:                                _step 10
wandb:                             _runtime 52
wandb:                

2021-02-10 14:26:35,669	WARNING util.py:142 -- The `start_trial` operation took 51.032 s, which may be a performance bottleneck.
== Status ==
Memory usage on this node: 11.3/15.3 GiB
Using FIFO scheduling algorithm.
Resources requested: 3/4 CPUs, 1/1 GPUs, 0.0/5.96 GiB heap, 0.0/2.05 GiB objects (0/1.0 accelerator_type:T4)
Result logdir: /home/ubuntu/s3/fine-tuning/hps_bart_para_wordembedray_logs/hps_bart_para_wordembed
Number of trials: 4/4 (1 RUNNING, 3 TERMINATED)
+--------------------+------------+-------+-------------------------------+-----------------+-------------+
| Trial name         | status     | loc   |   gradient_accumulation_steps |   learning_rate |   objective |
|--------------------+------------+-------+-------------------------------+-----------------+-------------|
| _inner_40086_00003 | RUNNING    |       |                             4 |           5e-05 |             |
| _inner_40086_00000 | TERMINATED |       |                             2 |           1e-05 |   


wandb: Waiting for W&B process to finish, PID 6977
wandb: Program failed with code 1.  Press ctrl-c to abort syncing.
wandb:                                                                                
wandb: Find user logs for this run at: /home/ubuntu/s3/fine-tuning/hps_bart_para_wordembedray_logs/hps_bart_para_wordembed/_inner_40086_00000_0_gradient_accumulation_steps=2,learning_rate=1e-05_2021-02-10_14-21-22/wandb/run-20210210_142605-23iega4g/logs/debug.log
wandb: Find internal logs for this run at: /home/ubuntu/s3/fine-tuning/hps_bart_para_wordembedray_logs/hps_bart_para_wordembed/_inner_40086_00000_0_gradient_accumulation_steps=2,learning_rate=1e-05_2021-02-10_14-21-22/wandb/run-20210210_142605-23iega4g/logs/debug-internal.log
wandb: Run summary:
wandb:            train/loss 3.8359
wandb:   train/learning_rate 5e-05
wandb:           train/epoch 0.16
wandb:                 _step 2
wandb:              _runtime 24
wandb:            _timestamp 1612967196
wandb: Run history:
wandb